In [1]:
import pandas as pd
import time
import datetime
from google.cloud import pubsub
import json
import apache_beam as beam
import os
print(beam.__version__)

No handlers could be found for logger "oauth2client.contrib.multistore_file"


2.1.0.dev


/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-10-10 18:42:49.314738. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [18]:
TIME_FORMAT = '%Y-%m-%d %H:%M:%S'
RUNNER = "Dataflow"
PROJECT = 'ksalama-gcp-playground'
DATASET = 'playground_ds'
TABLE = 'taxi_trips'
STG_BUCKET = 'stagging-ksalama-gcs-cloudml'
REGION = 'europe-west1'
TOPIC = 'taxi-trips'
SUBSCRIPTION='taxi-trips-sub'

/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-10-10 18:49:23.765724. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


## Submit Dataflow Stream Processing Job

In [3]:
pubsub_subscription = "projects/{}/subscriptions/{}".format(PROJECT,SUBSCRIPTION)
pubsub_topic = "projects/{}/topics/{}".format(PROJECT,TOPIC)

print(pubsub_subscription)
print(DATASET,TABLE)

def process_events(message):
    data_row = message.data
    #preform all yor data processing, transformation, calling external APIs, etc.
    return data_row
  
def run_taxi_trips_pipeline():
    
    job_name = 'ingest-taxi-trips-{}'.format(datetime.datetime.now().strftime('%y%m%d-%H%M%S'))
    print 'Launching Dataflow job {}'.format(job_name)
    print 'Check the Dataflow jobs on Google Cloud Console...'

    STG_DIR = 'gs://{}/taxi-fare'.format(STG_BUCKET)

    options = {
        'staging_location': os.path.join(STG_DIR, 'tmp', 'staging'),
        'temp_location': os.path.join(STG_DIR, 'tmp'),
        'job_name': job_name,
        'project': PROJECT,
        'streaming': True,
        'teardown_policy': 'TEARDOWN_ALWAYS',
        'no_save_main_session': True
      }


    opts = beam.pipeline.PipelineOptions(flags=[], **options)
    
    pipeline = beam.Pipeline(RUNNER, options=opts)
      
    (
      pipeline | 'Read data from PubSub' >> beam.io.ReadStringsFromPubSub(subscription=pubsub_subscription) 
               | 'Process message' >> beam.Map(process_events) # filter, window, group, aggregate 
               | 'Write to BigQuery' >> beam.io.WriteToBigQuery(project=PROJECT, dataset=DATASET,table=TABLE)
    )

    pipeline.run()

projects/ksalama-gcp-playground/subscriptions/taxi_trips-sub
('playground_ds', 'taxi_trips')


/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-10-10 18:42:50.874364. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [4]:
run_taxi_trips_pipeline()

Launching Dataflow job ingest-taxi-trips-171010-184251
Check the Dataflow jobs on Google Cloud Console...


/usr/local/lib/python2.7/dist-packages/apache_beam/utils/annotations.py:99: DeprecationWarning: options is deprecated since First stable release.. References to <pipeline>.options will not be supported
  warnings.warn(message, warning_type)
/usr/local/lib/python2.7/dist-packages/apache_beam/coders/typecoders.py:135: UserWarning: Using fallback coder for typehint: Any.
  warnings.warn('Using fallback coder for typehint: %r.' % typehint)
/usr/local/lib/python2.7/dist-packages/apache_beam/io/gcp/gcsio.py:113: DeprecationWarning: object() takes no parameters
  super(GcsIO, cls).__new__(cls, storage_client))
/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-10-10 18:42:51.619907. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


## Generate Data Points

In [5]:
csv_header_names = ['trip_datetime', 'pickup_dayofweek', 'pickup_hour','pickup_lon', 'pickup_lat', 'dropoff_lon', 'dropoff_lat', 'passenger_count', 'fare_amount']
header_names = ['trip_datetime', 'pickup_lon', 'pickup_lat', 'dropoff_lon', 'dropoff_lat', 'passenger_count', 'fare_amount']

dataset = pd.read_csv('data/train-data.csv', header=None, names=csv_header_names)[header_names]
def get_data_points(count=10):
  
    data_points = []
  
    instances = dataset.sample(n=count).values
  
    for row in instances:
        data_point = dict()
    
    for i in range(len(row)):
        data_point[header_names[i]] = row[i]
     
    data_points.append(data_point)
      
    return data_points

/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-10-10 18:43:03.343734. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


## Send Data Points to Pub/Sub

In [19]:
batch_size = 10
iterations = 10
sleep_time = 1

client = pubsub.Client()
topic = client.topic(TOPIC)

if not topic.exists():
    print ('Creating pub/sub topic {}...'.format(TOPIC))
    topic.create()

print ('Pub/sub topic {} is up and running'.format(TOPIC))
print("")

for i in range(iterations):

    data_points = get_data_points(batch_size)
    
    for data_point in data_points:

        source_id = str(abs(hash(str(data_point))) % (10 ** 10))
        source_timestamp = datetime.datetime.now().strftime(TIME_FORMAT)
        message = json.dumps(data_point)
        topic.publish(message=message, source_id = source_id, source_timestamp=source_timestamp)

    print("Batch {} was sent. Last Message was: {}".format(i, message))
    print("")

    time.sleep(sleep_time)

print("Done!")

Pub/sub topic taxi-trips is up and running

Batch 0 was sent. Last Message was: {"pickup_lon": -74.0049514771, "fare_amount": 7.5, "dropoff_lon": -73.9918060303, "passenger_count": 1, "pickup_lat": 40.7466087341, "trip_datetime": "2015-06-03 13:33:43", "dropoff_lat": 40.7645149231}

Batch 1 was sent. Last Message was: {"pickup_lon": -73.9788665771, "fare_amount": 33.33, "dropoff_lon": -73.8710021973, "passenger_count": 1, "pickup_lat": 40.7540245056, "trip_datetime": "2015-02-06 12:10:20", "dropoff_lat": 40.7741966248}

Batch 2 was sent. Last Message was: {"pickup_lon": -74.00728607180001, "fare_amount": 11.5, "dropoff_lon": -74.0135498047, "passenger_count": 1, "pickup_lat": 40.7434463501, "trip_datetime": "2015-02-15 01:39:24", "dropoff_lat": 40.7155380249}

Batch 3 was sent. Last Message was: {"pickup_lon": -73.9684677124, "fare_amount": 6.5, "dropoff_lon": -73.98252868649999, "passenger_count": 5, "pickup_lat": 40.7676086426, "trip_datetime": "2015-04-13 10:50:08", "dropoff_lat": 4

/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-10-10 18:49:32.305917. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


## Consume PubSub Topic 

In [20]:
client = pubsub.Client()
topic = client.topic(TOPIC)
subscription = topic.subscription(SUBSCRIPTION)
message = subscription.pull()

# print(message[0][1].source_timestamp)
print("source_id", message[0][1].attributes["source_id"])
print("source_timestamp:", message[0][1].attributes["source_timestamp"])
print("")
print(message[0][1].data)

('source_id', u'6121392541')
('source_timestamp:', u'2017-10-10 18:49:42')

{"pickup_lon": -73.9906539917, "fare_amount": 9.0, "dropoff_lon": -74.0055770874, "passenger_count": 1, "pickup_lat": 40.7510223389, "trip_datetime": "2015-05-28 19:04:25", "dropoff_lat": 40.725673675500005}


/usr/local/lib/python2.7/dist-packages/simplejson/encoder.py:291: DeprecationWarning: Interpreting naive datetime as local 2017-10-10 18:50:02.123470. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)
